In [15]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout, Input
from keras.losses import CategoricalCrossentropy 
import sys

def column_one_hot (dataframe, columns): 
    for column in columns:
        if column in dataframe:
            one_hot = pd.get_dummies(dataframe[column])
            dataframe = dataframe.drop(column,axis = 1)
            dataframe = pd.concat([dataframe, one_hot], axis=1)
    return dataframe

def drop_columns (dataframe, columns):
    for column in columns:
        if column in dataframe.columns:
            dataframe = dataframe.drop(column, axis=1)
    return dataframe

def drop_column (dataframe, column):
    if column in dataframe.columns:
        dataframe = dataframe.drop(column, axis=1)
    return dataframe

In [16]:
df = pd.read_csv('SAR_Data.csv')
df.shape

status = df['Status'].unique()
for x in status:
    print(x + " " + str(len(df[df['Status'] == x])))

print ("----------------------------------------------------------------")

maskOpen = ((df['Status'] == 'postponed') | (df['Status'] == 'partly open') |( df['Status'] == 'In creation'))
maskClosed = (df['Status'] == 'partly closed')
df.loc[maskOpen, 'Status'] = 'open'
df.loc[maskClosed, 'Status'] = 'closed'
df.loc[(df['Status'] == 'non applicable'), 'Status'] = 'not applicable'
status = df['Status'].unique()
for x in status:
    print(x + " " + str(len(df[df['Status'] == x])))

# postponed, partly open, in creation -> open;    partly closed -> closed

In creation 773
non applicable 4052
closed 6772
forwarded 1339
open 111
postponed 12
compliant 1413
partly closed 92
partly open 8
----------------------------------------------------------------
open 904
not applicable 4052
closed 6864
forwarded 1339
compliant 1413


In [ ]:
status = df['Status'].unique()
hauefigkeit = []
for x in status:
    hauefigkeit.append(len(df[df['Status'] == x]))
fig, ax = plt.subplots()
ax.pie(hauefigkeit, labels=status, autopct='%1.1f%%')

In [17]:
# falsche Ordnerstruktur in /RA Application Conditions/03_PG_OCS/Service and diagnostic systems
# fehlt ein Ordner bevor Module kommen, deshalb händisch eintragen

df.loc[df['Version'].str.contains('VICOS_S_D'), 'Product'] = 'VICOS_S_D'
df.loc[df['Version'].str.contains('VICOS_S_D'), 'Version'] = df['Version'].str[-5:]

In [ ]:
print(df['Product'].unique())
df[df['Product'] == "/"].shape

In [18]:
paths = df['Path'].unique()
accessDB = pd.read_xml("X:/File/DE/bwga024a_IMORA_RM/05_Process_Management/14_Metriken & KPI/KPI-Erhebung/KPI_01-04_General/Data/Input/Input_BWG_Combined_Access.xml")

for path in paths:
    try:
        if(path == "/ML Realization Projects Algeria"):
            result = accessDB.loc[(accessDB['Type'] == "Real") & (accessDB['Location'] == "BWG") & (accessDB['Offset'] == "/ML Realization Projects Algeria/20006_ML_BM_Boughezoul_MSila")].iloc[0]
        else:
            result = accessDB.loc[(accessDB['Type'] == "Real") & (accessDB['Location'] == "BWG") & ((accessDB['Offset'] == str(path)) | (accessDB['Offset'] == (str(path) + "/")))].iloc[0]           
    except:
        print(str(path) + " has no entry in the AccessDB!")

    mask = df['Path'] == str(path)
    df.loc[mask, 'Project_category'] = result['Project_category']
    df.loc[mask, 'BS'] = result['BS']
    df.loc[mask, 'RU'] = result['RU']
    df.loc[mask, 'ProjectYear'] = result['ProjectYear']
    df.loc[mask, 'section'] = result['section']
    df.loc[mask, 'Project_name'] = result['Project_name']
    df['ProductVersion'] = df["Product"].str.cat(df["Version"], sep = "-")

df['ProjectYear'] = df['ProjectYear'].astype('int')
df = df[['Text', 'Product', 'ProductVersion', 'Project_name', 'section', 'Project_category', 'BS', 'RU', 'ProjectYear', 'Status', 'Statement']]
#df = drop_column(df, 'Statement') # später wieder löschen!
df.head()
df.shape


(14572, 11)

In [ ]:
text = df['Text'].value_counts()
eins = 0
for anzahl in text:
    if anzahl == 1:
        eins += 1
print ("Anzahl Anwendungsregeln, welche nur einmal bewertet wurden: " + str(eins))
print ("Anteil am Datensatz: " + str(round(round(eins/df.shape[0], 4) * 100, 2)) + "%")
    
#text.head(10)
test = df.loc[df['Text'] == "Zur Anschaltung des Antriebes in der Außenanlage müssen Signalkabel nach VDE 0816/2 oder Kabel mit vergleichbaren Eigenschaften verwendet werden. Die Verlegevorschriften des Kabels sind einzuhalten."]
test.reset_index(inplace=True, drop=True)
test.head(17)

In [ ]:
print(df['Product'].unique())
testdf = df[df['Product'] == "/"]
testdf.shape
# '/57022-ML-HU-Szazhalombatta-Pusztaszabolcs' hat keinen Link auf RA Application Conditions sondern nur zu einem Modul in einem anderen Projekt aus Ungarn
# '/55525_ML_HU_KLBA' hat teilweise keinen Link auf RA Application Conditions

## Hoffentlich gefixt, Update: nur teilweise gefixt, 147 Objekte immernoch keinen Link auf RA Application Conditions

In [14]:
products = df['Product'].unique()
df_product = column_one_hot(df[['Product']], ['Product'])
projects = df['Project_name'].unique()
for project in projects:
    for product in products:
        df_product.loc[df['Project_name'] == project, product] = 1 if (df_product.loc[df['Project_name'] == project][product].sum()) >= 1 else 0
if (not (products[0] in df)):        
    df = df.join(df_product)
df = drop_column(df, 'Product')
df = column_one_hot(df, ['ProductVersion', 'Project_name', 'section',
       'Project_category', 'BS', 'RU', 'ProjectYear'])
df.head()

,Text,Status,Statement,/,AC100,ACM200,ACM300,AzS350U,CG ETCS Workstation,Clearguard TCM 100,...,2009,2011,2012,2013,2014,2015,2017,2018,2019,2020
0,The equipment described may only be used with...,open,TERMINATE Maintenance DCM 300 A6Z00050460843...,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,Equalize the charge before inserting the xCM....,open,TERMINATE Maintenance DCM 300 A6Z00050460843...,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,In order to avoid unnecessary aging of the spa...,open,TERMINATE Maintenance DCM 300 A6Z00050460843...,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,The DCM must be mounted on a mounting unit. If...,open,TERMINATE Covered by [REF PAC_DigIO_2980],0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,Dispose of faulty electronic circuit boards r...,open,TERMINATE Maintenance DCM 300 A6Z00050460843...,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [ ]:
df_training = df
text = "Zur Anschaltung des Antriebes in der Außenanlage müssen Signalkabel nach VDE 0816/2 oder Kabel mit vergleichbaren Eigenschaften verwendet werden. Die Verlegevorschriften des Kabels sind einzuhalten."
df_training = df_training.loc[df_training['Text'] == text]
df_training.reset_index(inplace=True, drop=True)

test_col = 12
test = drop_columns(df_training.iloc[[test_col]], ['Text', 'Status', 'Statement'])
df_training = df_training.drop(test_col)

In [ ]:
trainX = drop_columns(df_training, ['Status', 'Text', 'Statement'])
trainYStatus = drop_column(column_one_hot(df_training[['Text', 'Status']], ['Status']), "Text")
trainYStatement = drop_column(column_one_hot(df_training[['Text', 'Statement']], ['Statement']), "Text")

In [ ]:
modelStatus = Sequential()
modelStatus.add(Input(shape=trainX.shape[1]))
modelStatus.add(Dense(16, activation='relu'))
modelStatus.add(Dense(trainYStatus.shape[1], activation='softmax'))
modelStatus.summary()
modelStatus.compile(optimizer='adam',
              loss=CategoricalCrossentropy(),
              metrics=['accuracy'])

history = modelStatus.fit(trainX, trainYStatus,
                    batch_size=2,
                    epochs=50,
                    verbose=2,
                    validation_split=0.4)

modelStatement = Sequential()
modelStatement.add(Input(shape=trainX.shape[1]))
modelStatement.add(Dense(16, activation='relu'))
modelStatement.add(Dense(trainYStatement.shape[1], activation='softmax'))
modelStatement.summary()
modelStatement.compile(optimizer='adam',
              loss=CategoricalCrossentropy(),
              metrics=['accuracy'])

history = modelStatement.fit(trainX, trainYStatement,
                    batch_size=2,
                    epochs=50,
                    verbose=2,
                    validation_split=0.4)

In [ ]:
plt.plot(history.history['loss'], label = 'Training loss')
plt.plot(history.history['val_loss'], label = 'Validation loss')
plt.legend()

In [ ]:
predictionStatus = modelStatus.predict(test)
col = 0
for i in predictionStatus:
    for j in i:
        print (trainYStatus.columns[col] + " " + '{:.1%}'.format(j))
        col += 1

print ("-----------------------------------------------------")

predictionStatement = modelStatement.predict(test)
index_max = np.argmax(predictionStatement)
print (trainYStatement.columns[index_max] + " " + '{:.1%}'.format(predictionStatement[0][index_max]))
#col = 0
#for i in predictionStatement:
#    for j in i:
#        print (trainYStatement.columns[col] + " " + '{:.1%}'.format(j))
#        col += 1
#